# Data Plots

This notebook is designed to visualize the results of various configurations. It reads the results from CSV files, processes them, and generates plots to compare different metrics across configurations.

# Imports and dependencies

This section imports the required libraries for data manipulation and visualization.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Input variables

This section defines the base directory where the results are stored and if an llm model has been used during tests.

In [ ]:
base_dir = "output/prova/"
use_llm = True

# Searching for files

This section searches for all CSV files that contain results in the specified directory and its subdirectories. The files are expected to be named "results.csv".

In [ ]:
results_files = []
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith("results.csv"):
            results_files.append(os.path.join(root, file))

# Reading and processing the data

This section reads each results file into a DataFrame, extracts the configuration name from the file path, and appends it to a list. Finally, it concatenates all DataFrames into a single DataFrame for further analysis.

In [ ]:
df_list = []
for filepath in results_files:
    config_name = os.path.basename(os.path.dirname(filepath))
    df = pd.read_csv(filepath)
    df["config"] = config_name
    df_list.append(df)
    
combined_df = pd.concat(df_list, ignore_index=True)

# Metrics to plot

This section defines the metrics to be plotted based on whether an LLM model was used during the tests. If `use_llm` is set to `True`, additional metrics related to the LLM are included.

In [ ]:
metrics_to_plot = [
    "rougeL", "rougeL_golden", 
    "bleu", "bleu_golden", 
    "bleurt", "bleurt_golden", 
    "cosine_similarity", "cosine_similarity_golden",
    "longest_match", "longest_match_golden",
]

mean_metrics = combined_df.groupby("config")[metrics_to_plot].mean().reset_index()

# Plotting the metrics

This section reshapes the DataFrame for plotting and generates a bar plot to compare the specified metrics across different configurations. The plot is saved as an image file.

In [ ]:
long_df = mean_metrics.melt(id_vars="config", var_name="Metrics", value_name="value")

plt.figure(figsize=(12, 6))
sns.barplot(data=long_df, x="config", y="value", hue="Metrics")
plt.xticks(rotation=45, ha="right")
plt.title("Metrics Comparison")
plt.tight_layout()
plt.grid(True)


file_name = os.path.split(base_dir)[-1]
plt.savefig(os.path.join(base_dir, f"{file_name}_plot.png"))
plt.show()

In [ ]:
"""import json

responses_path = os.path.join(base_dir, "responses.json")

with open(responses_path, "r", encoding="utf-8") as f:
    responses = json.load(f)

def show_retrieved_docs(idx):
    r = responses[idx]
    print(f"Query [{idx}]: {r['query']}\n")
    print(f"Prediction: {r['prediction']}\n")
    print("Retrieved documents:")
    for i, doc in enumerate(r["retrieved_docs"]):
        print(f"  [{i+1}] {doc[:200]}...")
    print("="*80)

for idx in range(len(responses)):
    show_retrieved_docs(idx)"""

In [ ]:
import os

metrics_by_k_path = os.path.join(base_dir, "metrics_by_k.csv")
if os.path.exists(metrics_by_k_path):
    metrics_by_k = pd.read_csv(metrics_by_k_path)
    metrics = ["precision", "recall", "map", "ndcg"]

    plt.figure(figsize=(10, 6))
    for metric in metrics:
        plt.plot(metrics_by_k["k"], metrics_by_k[metric], marker="o", label=metric)
    plt.xlabel("k")
    plt.ylabel("Score")
    plt.title("Retrieval metrics by k")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(base_dir, "metrics_by_k_plot.png"), dpi=300)
    plt.show()
else:
    print(f"File {metrics_by_k_path} non trovato.")